In [1]:
lines = []
with open('text.txt', encoding='utf-8') as file:
    for line in file:
        lines.append(line.rstrip())

text = ' '.join(lines)

In [2]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    ORG,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

doc = Doc(text)

In [3]:
doc.segment(segmenter)
doc.tag_ner(ner_tagger)

In [4]:
org_spans = [span for span in doc.spans if span.type == ORG]
org_spans

[DocSpan(start=17, stop=27, type='ORG', text='Калашников', tokens=[...]),
 DocSpan(start=159, stop=162, type='ORG', text='ОПК', tokens=[...]),
 DocSpan(start=466, stop=476, type='ORG', text='Калашников', tokens=[...]),
 DocSpan(start=553, stop=558, type='ORG', text='Ижмаш', tokens=[...]),
 DocSpan(start=1005, stop=1011, type='ORG', text='Ростех', tokens=[...]),
 DocSpan(start=1142, stop=1152, type='ORG', text='Калашников', tokens=[...]),
 DocSpan(start=1341, stop=1381, type='ORG', text='Казённое предприятие 1.5\tСтолетие завода', tokens=[...]),
 DocSpan(start=1548, stop=1553, type='ORG', text='Ижмаш', tokens=[...]),
 DocSpan(start=1577, stop=1642, type='ORG', text='Концерн «Калашников» 2\tДеятельность 2.1\tРебренд..., tokens=[...]),
 DocSpan(start=2175, stop=2179, type='ORG', text='8\tСм', tokens=[...]),
 DocSpan(start=2243, stop=2250, type='ORG', text='Ижсталь', tokens=[...]),
 DocSpan(start=2795, stop=2827, type='ORG', text='Ижевский железоделательный завод', tokens=[...]),
 DocSpan(

In [5]:
from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.predicates import gram, in_, length_eq, is_upper, is_title, eq, type
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline
from yargy.tokenizer import (QUOTES, LEFT_QUOTES, RIGHT_QUOTES)

QUOTE = in_(QUOTES)
LEFT_QUOTE = in_(LEFT_QUOTES)
RIGHT_QUOTE = in_(RIGHT_QUOTES)

# Организация с формой управления, типом и названием
Organisation = fact(
    'Organisation',
    ['form', 'type', 'name']
)

# Форма управления организацией
FORM_DICT = ['ОАО', 'АО', 'ГК', 'ООО', 'НПО', 'ПАО']
FORM = morph_pipeline(FORM_DICT)

# Тип организации
TYPE_DICT = [
    'союз',
    'концерн',
    'федерация',
]
TYPE = morph_pipeline(TYPE_DICT)

# Наименование организации
OrgName = rule(
    gram('ADJF').repeatable().optional(),
    and_(
        gram('NOUN'),
        not_(in_(FORM_DICT)),
        not_(in_(TYPE_DICT))
    ),
    gram('ADJF').repeatable().optional()
)

gnc = gnc_relation()
ORGANISATION = rule(
    FORM.interpretation(
        Organisation.form
    ),
    LEFT_QUOTE.optional(),
    TYPE.interpretation(
        Organisation.type
    ).optional(),
    LEFT_QUOTE.optional(),
    OrgName.interpretation(
        Organisation.name
    ),
    RIGHT_QUOTE.optional(),
    RIGHT_QUOTE.optional()
).interpretation(
    Organisation
)

parser = Parser(ORGANISATION)

matches = []
for match in parser.findall(text):
    matches.append(match.fact)

matches

[Organisation(
     form='АО',
     type='Концерн',
     name='Калашников'
 ),
 Organisation(
     form='ОАО',
     type=None,
     name='КБАЛ'
 ),
 Organisation(
     form='ОАО',
     type=None,
     name='НИТИ'
 ),
 Organisation(
     form='ГК',
     type=None,
     name='Ростехнологии'
 ),
 Organisation(
     form='ОАО',
     type=None,
     name='Ижмаш'
 ),
 Organisation(
     form='НПО',
     type=None,
     name='Ижмаш'
 ),
 Organisation(
     form='НПО',
     type=None,
     name='Ижмаш'
 ),
 Organisation(
     form='ОАО',
     type='Концерн',
     name='Калашников'
 ),
 Organisation(
     form='НПО',
     type=None,
     name='Ижмаш'
 ),
 Organisation(
     form='АО',
     type='Концерн',
     name='Калашников'
 ),
 Organisation(
     form='ГК',
     type=None,
     name='Ростех'
 ),
 Organisation(
     form='ПАО',
     type=None,
     name='Ижевский механический завод'
 ),
 Organisation(
     form='ОАО',
     type=None,
     name='Мытищинский машиностроительный завод'
 ),
 Org

In [6]:
from yargy.predicates import is_title

# Награда - обладает типом (нп. орден) и названием
Award = fact(
    'Award',
    ['type', 'name']
)

AWARD_PRE = morph_pipeline([
    'награда',
    'награжден'
])

# Тип награды - орден или медаль с несколькими прилагательными
AWARD_TYPE_DICT = ['медаль', 'орден']
AWARD_TYPE = rule(
    gram('ADJF').repeatable().optional(),
    morph_pipeline(AWARD_TYPE_DICT)
)

# Название награды
AWARD_NAME = rule(
    LEFT_QUOTE.optional(),
    or_(
        rule(
            and_(is_title(), gram('ADJF')).repeatable(),
            gram('ADJF').optional().repeatable(),
            gram('NOUN'),
            gram('gent').repeatable().optional()
        ),
        rule(and_(is_title(), gram('gent'))),
    ),
    RIGHT_QUOTE.optional()
)

# Награда
AWARD = rule(
    or_(
        rule(
            AWARD_PRE,
            AWARD_TYPE.interpretation(
                Award.type.inflected()
            ),
            AWARD_NAME.interpretation(
                Award.name
            )
        ),
        rule(
            AWARD_PRE,
            AWARD_NAME.interpretation(
                Award.name
            )
        ),
    )
).interpretation(
    Award
)

parser = Parser(AWARD)

matches = []
for match in parser.findall(text):
    matches.append(match.fact)

matches

[Award(
     type='орден',
     name='Ленина'
 ),
 Award(
     type='орден',
     name='Трудового Красного Знамени'
 ),
 Award(
     type=None,
     name='Русской Православной церкви'
 ),
 Award(
     type='орден',
     name='Ленина'
 ),
 Award(
     type='орден',
     name='Трудового Красного Знамени'
 ),
 Award(
     type='орден',
     name='Октябрьской революции'
 )]

In [7]:
from yargy.predicates import length_eq, is_upper, type as ya_type

# Стрелковое оружие с типом, названием и калибром
Weapon = fact(
    'Weapon',
    ['type', 'name', 'caliber']
)

# Тип оружия
WEAPON_TYPE = morph_pipeline([
    'карабин',
    'винтовка',
    'автомат',
    'пистолет',
    'пулемет',
    'пулемёт',
    'ружьё',
])

INT = ya_type('INT')
# Аббревиатура
ABBR = and_(
    not_(length_eq(1)),
    is_upper()
)

# Наименование оружия - аббревиатура или существительное с заглавной буквы
# с числовыми обозначениями
WEAPON_NAME = rule(
    or_(
        ABBR,
        and_(
            is_title(),
            gram('NOUN'),
            not_(length_eq(1))
        )
    ),
    INT.optional(),
    rule(
        '-',
        or_(
            rule(INT),
            rule(ABBR),
        )
    ).optional(),
)

SEP = in_(',.')

NUM = rule(
    INT,
    rule(
        SEP,
        INT
    ).optional()
)

CALIBER_WORD = morph_pipeline(
    ['калибр', 'кал', 'кал.']
)
# Числовое обозначение калибра
CALIBER_VAL = rule(
    rule('.').optional(),  # .308
    NUM,  # 5.45
    rule(  # 9x19
        in_('xх×'),
        NUM
    ).optional(),
).interpretation(Weapon.caliber)

# Калибр
CALIBER = or_(
    rule(
        in_('(').optional(),
        CALIBER_WORD,
        CALIBER_VAL,
        rule('мм').optional(),
        in_(')').optional()
    ),
    rule(
        CALIBER_WORD.optional(),
        CALIBER_VAL,
        rule('мм')
    ),
    rule(
        CALIBER_VAL,
        CALIBER_WORD
    )
)

# Оружие
WEAPON = rule(
    WEAPON_TYPE.interpretation(
        Weapon.type.inflected()
    ),
    CALIBER.optional(),
    LEFT_QUOTE.optional(),
    WEAPON_NAME.interpretation(
        Weapon.name
    ),
    RIGHT_QUOTE.optional(),
    CALIBER.optional()
).interpretation(
    Weapon
)

parser = Parser(WEAPON)

matches = []
for match in parser.findall(text):
    matches.append(match.fact)

matches

[Weapon(
     type='винтовка',
     name='Мосина',
     caliber=None
 ),
 Weapon(
     type='карабин',
     name='Мосина',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Фёдорова',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Мосина',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Бердана',
     caliber=None
 ),
 Weapon(
     type='ружьё',
     name='ПСР',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Мосина',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='АВС-36',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Токарева',
     caliber=None
 ),
 Weapon(
     type='винтовка',
     name='Мосина',
     caliber=None
 ),
 Weapon(
     type='ружьё',
     name='Дегтярёва',
     caliber=None
 ),
 Weapon(
     type='ружьё',
     name='Симонова',
     caliber=None
 ),
 Weapon(
     type='пулемёт',
     name='Березина',
     caliber=None
 ),
 Weapon(
     type='пистолет',
     name='ТТ',
     c

In [8]:
[match for match in matches if match.caliber is not None]

[Weapon(
     type='пистолет',
     name='Лебедева',
     caliber='9×19'
 ),
 Weapon(
     type='ружьё',
     name='Сайга-12',
     caliber='12'
 ),
 Weapon(
     type='карабин',
     name='Сайга-МК',
     caliber='5,45×39'
 ),
 Weapon(
     type='винтовка',
     name='Биатлон-7',
     caliber='7,62'
 )]